In [10]:
import pandas as pd

from langchain_core.vectorstores import InMemoryVectorStore  

from langchain_openai import OpenAIEmbeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter  

from openai import OpenAI 

import requests

import json

from dotenv import load_dotenv
import os

# Подключаем все переменные из окружения
load_dotenv()
# Подключаем ключ для LLM-модели
LLM_API_KEY = os.getenv("LLM_API_KEY")
# Подключаем ключ для EMBEDDER-модели
EMBEDDER_API_KEY = os.getenv("EMBEDDER_API_KEY")

BASE_URL = 'https://openrouter.ai/api/v1'

client = OpenAI(
    base_url=BASE_URL,
    api_key=LLM_API_KEY,
)

questions = pd.read_csv('questions.csv')

corpus = pd.read_csv('train_data.csv')

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", base_url=BASE_URL, api_key=LLM_API_KEY)

In [27]:
class Chunker:
    def __init__(self, corpus):
        self.corpus = corpus
        self.splitter = RecursiveCharacterTextSplitter(  
            chunk_size=500,      # Maximum chunk size  
            chunk_overlap=100,    # Overlap between chunks for context  
            separators=["\n\n", "\n", " ", ""] 
        ) 
        self.chunks = self.chunk_corpus()

    def chunk_corpus(self):
        result = []
        for _, row in self.corpus.iterrows():
            tags = row['tags']
            annotation = row['annotation']
            text = row['text']
            result = result + self.chunk_doc(annotation, tags, text)
        chunks = [] 
        for c in result:
            if len(c) > 1000:
                chunks = chunks + self.splitter.split_text(text)
            else:
                chunks = chunks + [c]
        return chunks

    def chunk_doc(self, annotation, tags, text):
        chunks = self.chunk_text(text)
        tagged_annotation = self.tag_text(annotation, 'Annotation')
        tagged_tags = self.tag_text(tags, 'Tags')
        return ['\n'.join([tagged_annotation, tagged_tags, chunk]) for chunk in chunks]
        
    @staticmethod
    def tag_text(text, tag):        
        opening_tag = f'<{tag}>'
        closing_tag = f'</{tag}>'
        return f'{opening_tag}\n{text}\n{closing_tag}'

    @classmethod
    def tag_chunk_info(cls, chunk):
        header_closing = '</Header>'
        if header_closing not in chunk:
            return cls.tag_text(chunk, 'Information')
        header, info = chunk.split(header_closing)
        tagged_info = cls.tag_text(info, 'Information')
        return header_closing.join([header, tagged_info])
    
    @classmethod
    def chunk_text(cls, text):
        lines_to_replace = []
        for line in text.split('\n'):
            if '#' in line:
                lines_to_replace.append(line)
        text_ = text
        for line in lines_to_replace:
            tagged_header = cls.tag_text(line.replace('#', '').strip(), 'Header')
            tagged_header = '\n'.join(['<Delimiter>', tagged_header])
            text_ = text_.replace(
                line,
                tagged_header
            )
        result = [x.strip() for x in text_.split('<Delimiter>') if x.strip() not in ['\n', '', '\t']]
        result = [cls.tag_chunk_info(x) for x in result]
        return result

In [248]:
chunker = Chunker(corpus)

In [249]:
max(len(c) for c in chunker.chunks)

1000

In [250]:
vectorstore = InMemoryVectorStore.from_texts(  
    chunker.chunks,  
    embedding=embeddings, 
)

In [251]:
len(chunker.chunks)

35636

In [1]:
vectorstore.dump('my_vdb_new_chunks.db')

In [253]:
q = questions.head().iloc[2]['Вопрос']
q

'Как действовать при оспаривании незаконной продажи долга с учетом установленных сроков ответа на претензии?'

In [254]:
results = vectorstore.similarity_search(query=q, k=10)

In [262]:
results = vectorstore.similarity_search(query='Долг продали с нарушениями. Может оказаться, что банк, МФО или КПК действительно уступили ваш долг кому-то и не получили перед этим вашего разрешения. В этом случае вы имеете полное право отказаться от общения с новым кредитором и выплачивать заем прежнему. Стоит подать претензию организации, у которой вы брали в долг. Кредитор должен отреагировать за 15 рабочих дней. А если ему понадобятся от вас дополнительные документы — за 25 рабочих дней. Сроки прошли, а ответа нет или он кажется вам формальным — жалуйтесь в Банк России.', k=10)

In [263]:
merged_text = '\n'.join([doc.page_content for doc in results])

In [264]:
knowledge = Chunker.tag_text(merged_text, 'Knowledge')

In [23]:
OpenAI

openai.OpenAI

In [265]:
completion = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
    "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
  },
  extra_body={},
  model="x-ai/grok-3-mini",
  messages=[
    {
      "role": "user",
      "content": f"""Ответьте на вопрос, используя только релевантную предоставленную информацию. Убедитесь в том, что информация в тексте связана с вопросом. Не используйте не относящиеся к вопросу данные. Приведите ответ на русском языке.
Начните с краткого чек-листа (3–7 пунктов) основных шагов для ответа на вопрос, чтобы структурировать дальнейший ответ.
После формирования ответа проверьте, что все элементы из чек-листа раскрыты и информация соответствует предоставленным данным. При необходимости уточните и скорректируйте ответ.<Question>{q}</Question>{knowledge}"""
    }
  ]
)

# LLM Chunking

In [3]:
from typing_extensions import Annotated, TypedDict  

In [4]:
from langchain_openai import ChatOpenAI  

In [5]:
from typing import List

In [6]:
class Paragraph(TypedDict):
    title: str
    context: str
    problem: str 
    solution: str

In [7]:
class ParagraphedText(TypedDict):
    paragraphs: List[Paragraph]

In [8]:
from tqdm import tqdm

In [9]:
import asyncio
from tqdm.asyncio import tqdm_asyncio

In [51]:
import asyncio
from tqdm.asyncio import tqdm_asyncio

class LLMChunker:
    def __init__(self, model, corpus, max_concurrent=20):
        self.model = model.with_structured_output(ParagraphedText)
        self.corpus = corpus
        self.max_concurrent = max_concurrent
        self.semaphore = asyncio.Semaphore(max_concurrent)
        self.chunks = []
        self.failed_docs = []
        self.raw_paragraphs = []

    async def chunk_text(self, annotation, text, row_id=None):
        async with self.semaphore:  # Limits concurrency
            try:
                query = f"""
                    Разбей этот текст на один или несколько параграфов.
                    Требования:
                    - Используй только русский язык.
                    - Каждый параграф должен быть самостоятельным текстом, отвечающим на конкретный вопрос.
                    - Перед обработкой текста, начни с краткого чек-листа (3-7 пунктов), описывающего основные шаги, которые ты выполнишь.
                    - Для каждого параграфа создай отдельный JSON-объект со следующими полями:
                      title, context, problem, solution.
                    - Используй ТОЛЬКО информацию, предоставленную в исходном тексте.
                    Текст:
                    {annotation} 
                    {text}"""
                result = await asyncio.to_thread(self.model.invoke, query)
                self.raw_paragraphs.append(result)
                return result['paragraphs']
            except Exception as e:
                print(f"Error on doc {row_id}: {e}")
                self.failed_docs.append(row_id)
                return []

    @staticmethod
    def tag_text(text, tag):        
        return f'<{tag}>\n{text}\n</{tag}>'
        
    def dict_to_xml(self, doc_dict):
        return '\n'.join(self.tag_text(v, k) for k, v in doc_dict.items())

    async def chunk_corpus(self):
        tasks = [
            self.chunk_text(row['annotation'], row['text'], row.get('id'))
            for _, row in self.corpus.iterrows()
        ]
        results = await tqdm_asyncio.gather(*tasks, desc="Processing chunks")

        for result in results:
            for x in result:
                self.chunks.append(self.dict_to_xml(x))

In [52]:
model = ChatOpenAI(base_url=BASE_URL, model="x-ai/grok-3-mini", api_key=LLM_API_KEY)  

In [53]:
structured_model = model.with_structured_output(ParagraphedText)  

In [54]:
chunker = LLMChunker(model, corpus)

In [55]:
await chunker.chunk_corpus()

Processing chunks: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 350/350 [09:24<00:00,  1.61s/it]


In [56]:
my_chunks = chunker.chunks

In [57]:
my_chunks

['<title>\nКто такой самозанятый и кто может зарегистрироваться\n</title>\n<context>\nВ тексте объясняется, что самозанятый — это человек, который платит налог на профессиональный доход (НПД) без дополнительных налогов. Это подходит для россиян и граждан определенных стран, проживающих в РФ, даже для подростков с 14 лет с согласием родителей.\n</context>\n<problem>\nУ человека, как у Светланы, неофициальный доход, что затрудняет получение кредита или визы из-за отсутствия подтверждения дохода.\n</problem>\n<solution>\nЗарегистрироваться как самозанятый, чтобы легализовать доход и получить официальные справки, платя НПД по ставкам 4% от физических лиц и 6% от юридических.\n</solution>',
 '<title>\nСтавки налога на профессиональный доход\n</title>\n<context>\nСамозанятые платят налог в зависимости от источника дохода: 4% от физических лиц и 6% от юридических лиц или ИП, и это не изменится до 2028 года.\n</context>\n<problem>\nСветлана зарабатывает на уроках английского и должна платить н

In [58]:
# vectorstore = InMemoryVectorStore.from_texts(  
#     chunker.chunks,  
#     embedding=embeddings, 
# )

In [59]:
for i, chunk in enumerate(my_chunks):
    with open(f'./my_chunks_grok/chunk_{i}.txt', 'w', encoding='utf-8') as f:
        f.write(chunk)

In [ ]:
with open(f'./failed.txt', 'w') as f:
    f.write(str(chunker.failed_chunks))

In [179]:
print(chunker.chunks[10])

<context>
Выбор банка играет важную роль при получении кредита.
</context>
<problem>
Заемщики часто выбирают банк по неуместным критериям, таким как близость отделения или яркая реклама.
</problem>
<solution>
Изучите условия кредитов в нескольких организациях. Обратите внимание не только на процентную ставку, но и на другие параметры: срок кредитования, штрафы и пени за просрочку, необходимость страховки и ее стоимость.
</solution>
<title>
Не берите кредит в первом же банке
</title>
